In [1]:
import netsquid as ns
from netsquid.nodes import Node, DirectConnection
from netsquid.components import QuantumChannel, ClassicalChannel
from netsquid.protocols import NodeProtocol
from netsquid.components.models.delaymodels import FibreDelayModel

import numpy as np

In [2]:
class SourceProtocol(NodeProtocol):
    def __init__(self, node, n, GHZ):
        super().__init__(node)
        self.n = n
        self.ghz_state = GHZ
    def run(self):
        # creates n qubits and its state
        qbits = ns.qubits.create_qubits(self.n)
        if self.ghz_state:
            ns.qubits.combine_qubits(qbits)
            ns.qubits.operate(qbits[0], ns.H)
            for x in range(self.n-1):
                ns.qubits.operate([qbits[x], qbits[x+1]], ns.CNOT)
#         print("state prepared")
        # sends these states to party
        for x in range(self.n):
            self.node.ports[f'qout{x}'].tx_output(qbits[x])
#         print("qubits sent")

In [3]:
class PartyProtocol(NodeProtocol):
    def __init__(self, node, n=0, verifier=False):
        super().__init__(node)
        self.n = n
        self.verifier = verifier
    def run(self):
#         print("party protocol started")
        yield self.await_port_input(self.node.ports['qin'])
        state = self.node.ports['qin'].rx_input().items[0]
#         print("state recived")
        if self.verifier:
            total_sum=0.0
            recived_sum=0
            # generate random theta and send to party
            for x in range(1, self.n):
                theta = np.random.randint(2)
                total_sum += 0.5 if (theta==1) else 0
                self.node.ports[f'cout{x}'].tx_output(theta)
                yield self.await_port_input(self.node.ports[f'cin{x}'])
                recived_sum ^= self.node.ports[f'cin{x}'].rx_input().items[0]
            # measure for self state with total_sum theta and add to recived_sum
            if total_sum.is_integer():
                ns.qubits.operate(state, ns.H)
            else:
                total_sum+=.5
                ns.qubits.operate(state, ns.S)
                ns.qubits.operate(state, ns.Z)
                ns.qubits.operate(state, ns.H)
            m,p = ns.qubits.measure(state)
            recived_sum^=m
            print(total_sum, recived_sum)
            result = (recived_sum==(total_sum%2))
            if result==True:
                print("Verified, honest model")
            else:
                print("Not verified, dishonest model")
                
        else:
            yield self.await_port_input(self.node.ports['cin'])
            # theta is either 0(0 rad) or 1(pi/2 rad)
            theta = self.node.ports['cin'].rx_input().items[0]
            # measure in corrosponding basis and send result to verifier
            if theta==0:
                ns.qubits.operate(state, ns.H)
            elif theta==1:
                ns.qubits.operate(state, ns.S)
                ns.qubits.operate(state, ns.Z)
                ns.qubits.operate(state, ns.H)
            m,p = ns.qubits.measure(state)
            self.node.ports['cout'].tx_output(m)

In [4]:
#Define nodes in the network
n=4 # 1 source_node, 3 party_nodes(1 verifier_node inc)
source_node = Node(name="source")
party_nodes = []
for x in range(n):
    party_nodes.append(Node(name=f'party{x}'))

#Define channels
delay_model = FibreDelayModel()
distance = 400/1000 #Distance between nodes in km

#Quantum channel
s2p_qcs = []
for x in range(n):
    s2p_qcs.append(QuantumChannel(name = f's2p{x}_qc',length = distance, models = {"delay_model":delay_model}))

#Classical channels
v2p_ccs = [None]
for x in range(1,n):
    v2p_ccs.append(ClassicalChannel(f'v2p{x}_cc',length = distance, models = {"delay_model":delay_model}))
p2v_ccs = [None]
for x in range(1,n):
    p2v_ccs.append(ClassicalChannel(f'p{x}2v_cc',length = distance, models = {"delay_model":delay_model}))

#Add ports to nodes
for x in range(1,n):
    source_node.add_ports([f'qout{x}'])
    party_nodes[x].add_ports(['qin','cin','cout'])
    party_nodes[0].add_ports([f'cout{x}',f'cin{x}'])
source_node.add_ports(['qout0'])
party_nodes[0].add_ports(['qin'])

#Connect ports to channels
for x in range(n): 
    # connecting source_to_party
    source_node.ports[f'qout{x}'].connect(s2p_qcs[x].ports['send'])
    party_nodes[x].ports['qin'].connect(s2p_qcs[x].ports['recv'])
for x in range(1,n): 
    # connecting verifier_to_party
    party_nodes[0].ports[f'cout{x}'].connect(v2p_ccs[x].ports['send'])
    party_nodes[x].ports['cin'].connect(v2p_ccs[x].ports['recv'])
    # connecting party_to_verifier
    party_nodes[x].ports['cout'].connect(p2v_ccs[x].ports['send'])
    party_nodes[0].ports[f'cin{x}'].connect(p2v_ccs[x].ports['recv'])

In [5]:
# start protocols run_simulation stop protocols for GHZ state
for _ in range(100):
    ghz_source = SourceProtocol(source_node, n, True)
    O_source = SourceProtocol(source_node, n, False)
    party = [PartyProtocol(party_nodes[0], n, True)]
    for x in range(1, n):
        party.append(PartyProtocol(party_nodes[x]))
    ghz_source.start()
    for x in party:
        x.start()

    run_stats = ns.sim_run()
    # print(run_stats)

    ghz_source.stop()
    for x in party:
        x.stop()

1.0 1
Verified, honest model
0.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
0.0 0
Verified, honest model
2.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
0.0 0
Verified, honest model
2.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
2.0 0
Verified, honest model
0.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
2.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
0.0 0
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
0.0 0
Verified, honest model
1.0 1
Verified, honest model
2.0 0
Verified, honest model
1.0 1
Verified

In [6]:
# start protocols run_simulation stop protocols for |0> state on all qbits
for _ in range(100):
    ghz_source = SourceProtocol(source_node, n, True)
    O_source = SourceProtocol(source_node, n, False)
    party = [PartyProtocol(party_nodes[0], n, True)]
    for x in range(1, n):
        party.append(PartyProtocol(party_nodes[x]))
    O_source.start()
    for x in party:
        x.start()

    run_stats = ns.sim_run()

    O_source.stop()
    for x in party:
        x.stop()

2.0 0
Verified, honest model
0.0 1
Not verified, dishonest model
0.0 1
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
0.0 0
Verified, honest model
2.0 0
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
0.0 0
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
2.0 1
Not verified, dishonest model
1.0 0
Not verified, dishonest model
1.0 1
Verified, honest model
1.0 1
Verified, honest model
1.0 0
Not verified, dishonest model
1.0 0
Not verified, dishonest mod